## Aim:

We aim to see the evolvement of the temperature of a typical ride projected on the map of trajectory. Eventually we see the 
- path of the bike across the city
- temperature rise across the bike ride

### The beer box
This is our simulation of the bike-transportation box. The sensitive products (e.g. medicine), that have to be transported in low temperatures, is simulated usinga cold beer. 

<img style="float:left;transform: rotate(180deg); width:300px" src="pics/beer_box.JPG" />

In [178]:
import pandas as pd
import folium
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [282]:
df = pd.read_json('ride_data/bike_ride.json')
df = df.rename(columns = {'receivedAt':'timestamp',
                    'temp-inside':'t-in',
                    'temp-outside':'t-out'})

df['lat'] = df['lat'].apply(lambda x: None if x==0 else float(x))
df['lon'] = df['lon'].apply(lambda x: None if x==0 else float(x))
df = df.dropna()

# keep some datetime object in nice format
df['datetime'] = pd.to_datetime(df['timestamp'])
df['datetime'] = df['datetime'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M"))

df = df[['timestamp', 'lat', 'lon', 't-in', 't-out']]
df.index = df['timestamp'].values

# hack 
# reverse the temp because in this setup the cold beer, first cools down the box and then the heating process starts.
df['t-in'] = df['t-in'].values[::-1]
df['t-out'] = df['t-out'].values[::-1] # also t-out is actually inside the box in this setup

# and save it to csv
df.to_csv('ride_data/bike_ride.csv', index=False)

df.head(2)

,timestamp,lat,lon,t-in,t-out
2018-06-29T17:14:34.587Z,2018-06-29T17:14:34.587Z,47.372006,8.531828,17.698669,24.241638
2018-06-29T17:14:46.919Z,2018-06-29T17:14:46.919Z,47.371995,8.531871,17.799377,24.127197


## Visualize Heatmap across the ride 

In [301]:
lat0 = df['lat'].values[0]
lon0 = df['lon'].values[0]

In [300]:
# scale so that the change of temp is obvious
from sklearn.preprocessing import MinMaxScaler
df['t-in-normalized'] = MinMaxScaler().fit_transform(df['t-in'].values)

/Users/charilaostsarouchas/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/charilaostsarouchas/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [302]:
m = folium.Map([lat0, lon0], tiles='stamentoner', zoom_start=16)
plugins.HeatMap(df[['lat', 'lon', 't-in-normalized']].values.tolist()).add_to(m)
m

As we notice from the above map 
- the temperature rises across the ride (start bottom-left, end top-right)
- the hot in bottom-left corresponds to the starting/ending point where we touced the temperature sensor.
